In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from symspellpy.symspellpy import SymSpell, Verbosity
import json
import pandas as pd

Importación de la data

In [ ]:
# Cargar la tabla completa
# combined_reviews = pd.read_sql("SELECT * FROM combined_reviews", con=engine)

# # Filtrar en Python
# filtered_reviews = combined_reviews.groupby('Modelo estandar').filter(lambda x: x['comentario'].nunique() >= 200)

# # Guardar la tabla filtrada nuevamente en MySQL
# filtered_reviews.to_sql('filtered_reviews', con=engine, if_exists='replace', index=False)

In [ ]:
df_as= pd.read_csv(r"d:\Proyecto Web Scrapping\SQL Queries\filtered_reviews.csv")
print(df_as.info())  # Resumen del DataFrame
print(df_as['calificacion'].value_counts())  # Distribución de calificaciones

Pre-procesamiento del Texto

In [ ]:


# Descargar recursos necesarios de nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Configurar stopwords y lematización
stop_words = set(stopwords.words('spanish'))
lemmatizer = WordNetLemmatizer()

# Función para eliminar la frase completa y Diccionario de correcciones específicas
def eliminar_frase(text):
    return text.replace("(Esta reseña se ha recibido a través de una promoción.)", "")
correcciones = {
    "q": "que",
    "exelente": "excelente",
    "espectativas": "expectativas",
    "celu": "celular",
    "bno": "bueno",
    "auriculare": "auriculares",
    "cel": "celular",
    "sultra": "ultra",
    "aprox": "aproximadamente",
    "mobil": "movil",
    "amooo": "amo",
    "perfeccoon": "perfeccion",
    "camras": "camaras",
    "escojer": "escoger",
    "encamta": "encanta",
    "aser": "hacer",
    "calentarce": "calentarse",
    "parese": "parece",
    "nuy": "muy",
    "okey": "okay",
    "espetativas": "expectativas",
    "pq": "por que",
    "mut": "muy",
    "ecxelente": "excelente",
    "execelente": "excelente",
    "bna": "buena",
    "redimiento": "rendimiento",
    "qe": "que",
    "encata": "encanta",
    "sobrecalienta": "calienta",
    "buenardo": "bueno",
    "demaciado": "demasiado",
    "(Esta reseña se ha recibido a través de una promoción.)": "",
}
# Función para aplicar correcciones específicas
def aplicar_correcciones(text):
    tokens = text.split()  # Dividir el texto en palabras
    corrected_tokens = [correcciones.get(token, token) for token in tokens]  # Reemplazar si hay corrección
    return ' '.join(corrected_tokens)  # Volver a unir las palabras


# Stopwords personalizadas
important_negatives = {"no", "nunca", "sin", "muy", "poco", "bastante", "mas", "dos", "mucho"}
stop_words = stop_words - important_negatives

# Función para eliminar tildes sin alterar estructura
def remove_accents(text):
    accents = {
        'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
        'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U'
    }
    for accented_char, unaccented_char in accents.items():
        text = text.replace(accented_char, unaccented_char)  # Reemplazar cada carácter
    return text


# Función de limpieza completa
def clean_text_with_corrections(text):
    text = eliminar_frase(text)  # Eliminar la frase específica
    text = remove_accents(text)  # Eliminar tildes
    text = aplicar_correcciones(text)  # Aplicar correcciones específicas
    text = re.sub(r'\W', ' ', text)  # Eliminar caracteres no alfanuméricos
    text = re.sub(r'\s+', ' ', text).strip()  # Quitar espacios extra
    text = re.sub(r'\d+', '', text)  # Eliminar números
    tokens = word_tokenize(text.lower())  # Tokenizar
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]  # Lematización y filtro
    return ' '.join(tokens)

# Rellenar valores NaN en la columna 'comentario' con una cadena vacía
df_as['comentario'] = df_as['comentario'].fillna('')

# Aplicar al DataFrame
df_as['comentario_limpio'] = df_as['comentario'].apply(clean_text_with_corrections)

# Verificar cambios
print("Primeros comentarios limpios en el DataFrame de prueba:")
print(df_as[['comentario', 'comentario_limpio']].head(10))





